In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

`bbc.mtx` looks like this:

```
%%MatrixMarket matrix coordinate real general
9635 2225 286774
1 1 1.0
1 7 2.0
1 11 1.0
1 14 1.0
1 15 2.0
1 19 2.0
...
```

Together with the `mtx` extension, this should be a clue that `MatrixMarket` is a file format. A quick Google search turns up the `mmread` function:

In [ ]:
from scipy.io import mmread

In [ ]:
m = mmread('bbc.mtx')
m

Ok, so `m` has 9635 rows, one for each term in `bbc.terms`. And it has 2225 columns, each corresponding to a document listed in `bbc.docs`.

Now let's get going with NMF.

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
nmf = NMF(n_components=5, init='random', random_state=0)

In [ ]:
w = nmf.fit_transform(m)

In [ ]:
w.shape

The transformed `w` is a relationship between *terms* and *topics*. The implementation is "thinking" of the terms as the important data; to it the documents are just a coordinate system for expressing them.

But this is just an implementation quirk. Mathematically, there's no preferential treatment to rows vs columns. And we can easily get to the "other half" of the decomposition:

In [ ]:
h = nmf.components_
h.shape

NMF is an approximation, fitting more closely as the number of components increases. Let's have a look at how closely we fit in this case. First we can recover an estimate of the original `m` matrix:

In [ ]:
mhat = w.dot(h)

Now we can do a plot of predicted vs residual values, just as we do for regression. The matrix is pretty big, but we can still do this easily for the nonzeros:

In [ ]:
from scipy import sparse

i,j,v = sparse.find(m) 
vhat = [mhat[ii,jj] for (ii,jj) in zip(i,j)]
plt.scatter(vhat, v-vhat, alpha=0.1)
plt.xlabel("NMF Approximated value")
plt.ylabel("Residual");

In [ ]:
pred = w.argmax(1)

NMF gave us 5 topics; let's see if we can summarize these in terms of the vocabulary terms. First we'll read in the terms.

In [ ]:
with open('bbc.terms') as f:
     terms = f.readlines()

There's an annoying `newline` on the end of each, but that's easy to get rid of:

In [ ]:
terms = [t.split()[0] for t in terms]

In [ ]:
topic_words = []
for r in w.T:
    a = sorted([(v,i) for i,v in enumerate(r)],reverse=True)[0:7]
    topic_words.append([terms[e[1]] for e in a])

In [ ]:
len(topic_words)

In [ ]:
topic_words

The original data had 5 topics, as listed in `bbc.docs`. 

```
Business
Entertainment
Politics
Sport
Tech
```

In "real life", we would have found a way to use these to inform the model. But for this little demo, we can just compare the recovered topics to the original ones. And they seem to match reasonably well. The order is different, which is to be expected in this kind of model.